In [11]:
import pandas as pd
import folium
from folium.plugins import HeatMap

nypd_data_cleaned = pd.read_csv("/Users/shashankhmg/Documents/AXA-Casestudy/Data-Science-Challenge/data/processed/nypd_data_cleaned.csv")

citibike_stations = pd.read_csv("/Users/shashankhmg/Documents/AXA-Casestudy/Data-Science-Challenge/data/processed/cleaned_citibike_data.csv")


/var/folders/tw/yc01l_99601d14ks8nxwg1dr0000gn/T/ipykernel_89092/2503581617.py:7: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  citibike_stations = pd.read_csv("/Users/shashankhmg/Documents/AXA-Casestudy/Data-Science-Challenge/data/processed/cleaned_citibike_data.csv")


In [12]:
nypd_data_cleaned.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,...,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2
0,11/01/2023,1:29,BROOKLYN,11230.0,40.621790,-73.970024,"(40.62179, -73.970024)",OCEAN PARKWAY,1.0,0.0,...,0,0,0,1,0,Unspecified,Unspecified,4675373,Moped,Sedan
1,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",Unknown,0.0,0.0,...,0,0,0,0,0,Unspecified,Unknown,4456314,Sedan,Unknown
2,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,0.0,0.0,...,0,0,0,0,0,Unknown,Unknown,4486609,Unknown,Unknown
3,12/14/2021,8:17,BRONX,10475.0,40.868160,-73.831480,"(40.86816, -73.83148)",Unknown,2.0,0.0,...,0,0,0,2,0,Unspecified,Unspecified,4486660,Sedan,Sedan
4,12/14/2021,21:10,BROOKLYN,11207.0,40.671720,-73.897100,"(40.67172, -73.8971)",Unknown,0.0,0.0,...,0,0,0,0,0,Driver Inexperience,Unspecified,4487074,Sedan,Unknown


In [13]:
citibike_stations.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,56BD148A05E26915,electric_bike,2025-01-01 22:19:06.324,2025-01-01 22:23:32.899,W 36 St & 7 Ave,6483.06,W 24 St & 7 Ave,6257.03,40.752149,-73.989539,40.744876,-73.995299,member
1,700CCCDF00C08077,electric_bike,2025-01-09 15:20:43.991,2025-01-09 15:26:19.137,Broadway & E 19 St,5980.11,W 24 St & 7 Ave,6257.03,40.738290,-73.990060,40.744876,-73.995299,member
2,B1A3FAFA5FE651CE,electric_bike,2025-01-01 12:54:26.571,2025-01-01 13:03:03.565,Jackson Ave & 46 Rd,6203.02,Crescent St & Broadway,6827.11,40.745248,-73.947333,40.763359,-73.928647,member
3,9CB0F28054BA5EBC,electric_bike,2025-01-14 13:43:59.151,2025-01-14 13:49:04.543,Broadway & E 19 St,5980.11,W 24 St & 7 Ave,6257.03,40.738290,-73.990060,40.744876,-73.995299,member
4,A1B8FDD4FAF0C2B2,electric_bike,2025-01-13 07:20:37.176,2025-01-13 07:29:18.015,Willis Ave & E 143 St,7798.02,1 Ave & E 110 St,7522.02,40.812299,-73.920370,40.792327,-73.938300,member


In [14]:
# Filter accidents where a cyclist was injured or killed
cyclist_accidents = nypd_data_cleaned[
    (nypd_data_cleaned['NUMBER OF CYCLIST INJURED'] > 0) | 
    (nypd_data_cleaned['NUMBER OF CYCLIST KILLED'] > 0)
]

# Extract accident locations for heatmap
heatmap_data = cyclist_accidents[['LATITUDE', 'LONGITUDE']].dropna().values.tolist()
heatmap_data

[[40.687534, -73.9775],
 [40.767242, -73.986206],
 [40.59207, -73.96299],
 [40.640835, -73.98967],
 [40.76175, -73.96899],
 [40.736614, -73.9951],
 [40.730145, -73.99781],
 [40.754707, -73.99164],
 [40.87151, -73.87057],
 [40.76551, -73.97795],
 [40.724846, -73.999],
 [40.732006, -73.95795],
 [40.634007, -74.07516],
 [40.731613, -73.9545],
 [40.868164, -73.90054],
 [40.87037, -73.87052],
 [40.74958, -73.86541],
 [40.844105, -73.923065],
 [40.69484, -73.98391],
 [40.75868, -73.87552],
 [40.715702, -73.95579],
 [40.69349, -73.97917],
 [40.731422, -73.94641],
 [40.68836, -73.96444],
 [40.700485, -73.93637],
 [40.827824, -73.91934],
 [40.696983, -73.935234],
 [40.855873, -73.79149],
 [40.758156, -73.89627],
 [40.73444, -73.72179],
 [40.65319, -73.96622],
 [40.633514, -74.026955],
 [40.581745, -73.95384],
 [40.734863, -73.862656],
 [40.67506, -73.98139],
 [40.843822, -73.892494],
 [40.721474, -73.98383],
 [40.659832, -73.897606],
 [40.595802, -73.764656],
 [40.71082, -73.96853],
 [40.68249,

In [17]:
# Create a base map centered around NYC
nyc_map = folium.Map(location=[40.75, -73.98], zoom_start=12)

# Add accident heatmap layer
HeatMap(heatmap_data, radius=10, blur=15, max_zoom=1).add_to(nyc_map)

# Add CitiBike stations as markers
for index, row in citibike_stations.iterrows():
    folium.CircleMarker(
        #station lat and long are not directly given but obvi the start lat and long is the station lat and long
        location=[row['start_lat'], row['start_lng']],
        radius=3,  # Small circle to represent station
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7,
        popup=f"Station: {row['start_station_name']}"
    ).add_to(nyc_map)

# Save the map
nyc_map.save("/Users/shashankhmg/Documents/AXA-Casestudy/Data-Science-Challenge/heatmaps/citibike_accident_heatmap.html")
print(" Heatmap with CitiBike stations saved as 'citibike_accident_heatmap.html'")
